# SCC-DFTB SK-generation 

Input: DFTB_DB.db - ASE database with coordinates, energies and forces

Output: Slater-Koster files for each pair of elements stored in ./SKF/


Input for the Slater-Koster generation are taken from "$PARAUTOMATIK/pgm/SK_def/skdef.hsd". Modify this file if your elements does'nt exist.   


In [ ]:

PARAUTOMATIK_PATH='WRITE-Your-PATH-PARAUTOMATIK-Here'   # Add your installation path here

In [ ]:
import os, sys, warnings
import glob
import re
from shutil import copyfile, copytree
import itertools
import numpy as np
from ase.data import atomic_numbers, atomic_names, atomic_masses, covalent_radii, vdw_radii


base_dir=os.getcwd()  # You should be in your production folder now. 
print('Base dir is:', base_dir) 

warnings.filterwarnings('ignore')
sys.path.append(PARAUTOMATIK_PATH+'/pgm/skprogs/opt/lib/python3.9/site-packages/')

import argparse
import numpy as np
import sktools.common as sc
import sktools.skgen as skgen
from sktools.skdef import Skdef
from sktools import PACKAGE_VERSION
import sktools.sk_util as sk_util
cwd = os.getcwd()
# !source /home/broqvist/Desktop/parautomatik/pgm/skprogs/opt/bin/skprogs-activate.sh should be issued before starting jupyter-lab
skdef_all = PARAUTOMATIK_PATH+'/pgm/SK_def/skdef.hsd'  # parameter file for the data generation
SCRIPTNAME = sc.get_script_name()

# make the directory for the SK-generation and move in to that directory
if not os.path.isdir("SKF/"):
    os.mkdir("SKF/")

os.chdir("SKF/")
print('You are here',os.getcwd())

In [ ]:
# automatic search for elements in database
print('You are here',os.getcwd())

import ase.db as db
DFT_DB=db.connect("../DFT_DB.db")
elements=[]
for row in DFT_DB.select():
    structure=row.toatoms()
    a=structure.get_chemical_symbols()
    for i in a: 
        if i not in elements:
            elements.append(i)

elem=''
for i in elements:
    if elem=='':
        elem=i
    else:
        elem=elem+','+i
print(elem)        

In [ ]:
#set up input for skprogs and run skprogs generating the Slater-Koster files with a dummy rep
parser, subparsers = sk_util.get_parser_and_subparser_container()
sk_util.setup_parser_main(parser)
onecnt_common = sk_util.get_onecnt_common_parser()
sk_util.setup_parser_atom(subparsers, onecnt_common, sk_util.run_atom)
sk_util.setup_parser_denscomp(subparsers, onecnt_common, sk_util.run_denscomp)
sk_util.setup_parser_wavecomp(subparsers, onecnt_common, sk_util.run_wavecomp)
twocnt_common = sk_util.get_twocnt_common_parser()
sk_util.setup_parser_twocnt(subparsers, twocnt_common, sk_util.run_twocnt)
sk_util.setup_parser_sktable(subparsers, twocnt_common, sk_util.run_sktable)

class Args:
    includedirs=[] 
    configfiles=[skdef_all]
    builddir='_build' 
    onecnt_binary='slateratom' 
    twocnt_binary='sktwocnt'
    loglevel='info' 
    element1=elem
    element2=elem 
    dummyrep=True 
    outdir='.'
    func=sk_util.run_sktable

        
args=Args()

#run 
args.func()